In [68]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct

In [88]:
#reading in data agains
dfp_click_data = spark.read.parquet("gs://ds-taste-dfp/aggregated_data/click_aggregated_by_date/year=*/*/*.parquet")

In [89]:
# aggregate all the impression and clicks by campaigns and userid 
dfp_click_data_agree = dfp_click_data.groupby('orderid','line_itemid','adid','IABTier1Categorization').agg(func.sum("impressions").alias('Impressions'),func.sum("clicks").alias('Clicks'))


In [90]:
orders_travel = dfp_click_data_agree.where((col('orderid') == '2205238061') | (col('orderid') == '2204480300') | (col('orderid') == '2196314095') | (col('orderid') == '2197861368') | (col('orderid') == '2250263757') | (col('orderid') == '2256707010') | (col('orderid') == '2260774529') | (col('orderid') == '2266410121')  | (col('orderid') == '2250360384')  | (col('orderid') == '2256702195')  | (col('orderid') == '2260790765')    | (col('orderid') == '2266260188'))

In [91]:
orders_shopping = dfp_click_data_agree.where((col('orderid') == '2180152030') | (col('orderid') == '2204480300'))

In [92]:
# orders_travel.groupby('orderid').count().show()

In [93]:
# orders_shopping.groupby('orderid').count().show()

In [94]:
#samples where atleast there is one click and removing null adids 
sample = orders_travel.where(col('Clicks') > 0 )
new_sample = sample.where(col('adid') != '')

In [95]:
#samples where atleast there is one click and removing null adids 
sample_shopping = orders_shopping.where(col('Clicks') > 0 )
new_sample_shopping = sample_shopping.where(col('adid') != '')

In [96]:
# sample_shopping.count()

In [97]:
# new_sample_shopping.distinct().count()

In [98]:
from hashlib import sha1
import binascii
from pyspark.sql.types import StringType
import re

SALT_A = "77EfzyR%iZtyTFAK7lyZ_X=B7e^b1atUx%W+Rx0A3ja~b5yjGL-q==po%nkL"

def adid_to_ppid(adid):
    sha1hash = []
    pattern = re.compile("^[a-zA-Z0-9\-]{8}-[a-zA-z0-9]{4}-[a-zA-z0-9]{4}-[a-zA-z0-9]{4}-[a-zA-z0-9]{12}")
    if re.match(pattern,adid) is not None:
        text = adid + SALT_A
        sha1_dig = sha1(text).digest()
        sha1hash = [ord(c) for c in sha1_dig]
    return binascii.hexlify(bytearray(sha1hash))

adid_ppid_udf = udf(adid_to_ppid, StringType())


new_sample = new_sample.withColumn('ppid',adid_ppid_udf('adid'))
new_sample_shopping = new_sample_shopping.withColumn('ppid',adid_ppid_udf('adid'))

In [99]:
new_sample.select('ppid').distinct().coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").save("gs://ds-url-catag/mark_request/mar20/travel_ppids/")

In [100]:
new_sample_shopping.select('ppid').distinct().coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").save("gs://ds-url-catag/mark_request/mar20/shopping_ppid/")

In [101]:
new_sample_shopping.select('adid').distinct().coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").save("gs://ds-url-catag/mark_request/mar20/shopping_adid/")